In [1]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM,AutoTokenizer
from modeling_backpack_gpt2 import BackpackGPT2LMHeadModel
from configuration_backpack_gpt2 import BackpackGPT2Config
import json

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load JSON file
with open('/mnt/d/EE/Term6/Projects/LLM/BP/config.json', 'r') as file:
    conf = json.load(file)

config = BackpackGPT2Config(**conf) 
model = BackpackGPT2LMHeadModel(config)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# 1536 
checkpoint_path = '/mnt/d/EE/Term6/Projects/LLM/BP/pytorch_model.bin'
state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Filter the external state_dict to retain only the keys present in the model's state_dict
filtered_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()}

model.load_state_dict(state_dict=filtered_state_dict)
model.to(device)

print('Model is Loaded!')
model.eval()


Model is Loaded!


BackpackGPT2LMHeadModel(
  (backpack): BackpackGPT2Model(
    (gpt2_model): GPT2Model(
      (wte): Embedding(50264, 768)
      (wpe): Embedding(512, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv

In [2]:

input_text = "Hi, What are you doing?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# input = torch.randint(0, 50264, (1, 512), dtype=torch.long)
torch_out = model(
    input_ids,
    position_ids=None,
)

out = torch.nn.functional.softmax(torch_out.logits, dim=-1) 

print("Output shapes: ",out.shape) 
print("Senses shapes: ",torch_out.senses.shape)



torch.Size([1, 7, 7]) torch.Size([1, 16, 7, 768]) asasasasa
Output shapes:  torch.Size([1, 7, 50264])
Senses shapes:  torch.Size([1, 16, 7, 768])
